# サンプルアプリのセットアップ

BBQL(BigBaBy Quick Learning) のアプリケーションをセットアップします。本アプリケーションで使用している主な OSS は以下の通りです。

- LangChain: 0.2.x
- FAISS: 1.8.0
- Langfuse: v2.75.2

## 必要ライブラリのインストール

In [ ]:
%pip install -r ../requirements.txt

## ベクトルデータベース（FAISS） のセットアップ

`./docs/*.txt` に格納されているレシピデータに対して、埋め込み表現（Embeddings）を取得し、そのデータを FAISS に格納します。  
埋め込み表現の取得には、Cohere から提供されている `embed-multilingual-v3.0` というモデルを使用します。

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

# ../.env を読み込みし、必要な環境変数を取得します
_ = load_dotenv(find_dotenv())

cohere_api_key = os.getenv("COHERE_API_KEY")

埋め込みに使うモデルを定義します。

In [ ]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    cohere_api_key=cohere_api_key,
    model="embed-multilingual-v3.0"
)

ベクトルデータベース(FAISS)を宣言します。

In [4]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

`../docs/*.txt` に格納されているテキストデータを読み込み、LangChain の Document へ変換します。

In [5]:
import glob
from langchain.document_loaders import TextLoader

files = glob.glob("../docs/*.txt")
documents = []

for file in files:
    loader = TextLoader(file_path=file)
    document = loader.load()
    documents.extend(document)

読み込んだデータを FAISS に格納します。

In [ ]:
vector_store.add_documents(documents=documents)

自然言語を用いた類似度検索を行います。

In [ ]:
result = vector_store.similarity_search_with_score(query="カルビクッパ")
print(result)

## チャットモデル(cohere.command-r-plus) + ベクトルデータベース(FAISS) を用いたシンプルな RAG 構成

自然言語で回答を生成するために、LLM のモデルを定義します。  
ここでは、Cohere の Command R+ を使用します。

In [8]:
from langchain_cohere.chat_models import ChatCohere

chat = ChatCohere(
    cohere_api_key=cohere_api_key,
    model="command-r-plus"
)

Stream (LLMでトークンが出力されるたびに順次クライアントに返却する) 形式でトークンを生成します。

In [ ]:
response = chat.stream("カルビクッパってどうやって作るのでしょうか？")

for chunk in response:
    print(chunk.content, end="")

シンプルな　RAG 構成で直接 LLM を使う場合と振る舞いがどう変わるのか確認します。

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

retriever = vector_store.as_retriever()

prompt_template = PromptTemplate.from_template(template="""
以下のコンテキストに基づいて質問に対する回答を作成してください。

## コンテキスト

{context}

## 質問

{question}
""")

chain = (
    {"question": RunnablePassthrough(), "context": retriever}
    | prompt_template
    | chat
    | StrOutputParser()
)

response = chain.stream("カルビクッパってどうやって作るのでしょうか？")

for chunk in response:
    print(chunk, end="")